# Desarrollo 

In [1]:
# Librerias
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import torch
import os
import time
print(torch.cuda.is_available())
from torch.utils.data import DataLoader, Subset
import sklearn.model_selection
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import pandas as pd
import torchaudio
import datetime
from codes import utils
from codes import metric as mt
from codes.dataset2 import WetSoundDataset
from sklearn.model_selection import KFold
import torch.nn as nn
from torch.nn.functional import sigmoid
from codes.models import *
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

True


2022-06-24 10:24:42.920329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-24 10:24:42.920347: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
rm -rf ./tmp/

In [3]:
main = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

## 1. Importación y pre-procesamiento del dataset

In [4]:
N_FOLDS = 4
AUDIO_DIR = os.path.join(main,"features/train_set/")
SAMPLE_RATE = 44100
LEN_SEC = 300
LEN_SAMPLES = LEN_SEC*SAMPLE_RATE
NUM_SAMPLES = 1365   # 1292 #Cantidad de secuencias , no largo de secuencia 12920/40
__class_labels = {
'perro'    : 0,
'rana'     : 1,
'lluvia'   : 2,
'motor'    : 3,
'ave'      : 4
}
N_FFT = 2048
HOP = int(N_FFT/2)
N_MELS = 40
LEN_MBE = round(LEN_SAMPLES/int(HOP*NUM_SAMPLES))
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")

Using device cuda


#### Dataloaders

In [5]:
wet = WetSoundDataset(AUDIO_DIR,
                        LEN_MBE,
                        NUM_SAMPLES,
                        __class_labels,
                        device)
print(f"There are {len(wet)} samples in the dataset.")

There are 5760 samples in the dataset.


# SEDnet

In [6]:
model = SEDnet(n_class=len(__class_labels),stereo=True)
print(model)

SEDnet(
  (CNN1): Sequential(
    (0): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(1, 5), stride=(1, 5), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=True)
  )
  (CNN2): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.5, inplace=True)
  )
  (CNN3): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=

/home/vpoblete/.conda/envs/sedtorch/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


## 4. Entrenamiento de la red convolucional

In [7]:
max_epochs = 500

In [8]:
kfold = KFold(n_splits=N_FOLDS, shuffle=True)
# K-fold Cross Validation model evaluation
%tensorboard --logdir runs
for fold, (train_ids, val_ids) in enumerate(kfold.split(wet)):
    torch.manual_seed(12345) # Inicialización
    # Print
    print(f'FOLD {fold+1}')
    print('--------------------------------')

    kfold_trainset = Subset(wet,train_ids)
    kfold_valset = Subset(wet, val_ids)

    trainloader = DataLoader(
                    kfold_trainset, 
                    batch_size=32, shuffle=True)
    valloader = DataLoader(
                    kfold_valset,
                    batch_size=64,shuffle=False)
    display(len(trainloader),len(valloader))
    
    
    
    model = SEDnet(n_class=len(__class_labels),stereo=True)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr = 1e-3)
    criterion = torch.nn.BCELoss(reduction="mean")
    
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

    writer = SummaryWriter('runs/sednet_trainer_{}'.format(timestamp))
    epoch_number = 0

    best_f1 = 0.
    best_er = 1e15
    F1 = False
    ER = False if F1 else True
    cnt = 0
    patience = int(max_epochs*0.2)

    for epoch in range(max_epochs):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train()
        running_tloss = 0.0
        for i,tdata in enumerate(trainloader):
            tinputs,tlabels = tdata
            
            optimizer.zero_grad()
            toutputs = model.forward(tinputs)
            tloss = criterion(toutputs,tlabels)
            tloss.backward()
            optimizer.step()
            running_tloss +=tloss.item()
        avg_tloss = running_tloss /(i+1)

        # We don't need gradients on to do reporting
        model.train(False)

        running_vloss = 0.0
        with torch.no_grad():
            for j, vdata in enumerate(valloader):
                vinputs, vlabels = vdata
                voutputs = model.forward(vinputs)
                vloss = criterion(voutputs, vlabels)
                vf1 = mt.f1_score(voutputs,vlabels)
                ver = mt.error_rate(voutputs,vlabels)
                running_vloss += vloss
                
        avg_vloss = running_vloss / (j + 1)
        print('LOSS train {} valid {}'.format(avg_tloss, avg_vloss), end='\r')
        torch.cuda.empty_cache()
        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                        { 'Training' : avg_tloss, 'Validation' : avg_vloss },
                        epoch_number + 1)
        writer.add_scalars('F1 - score',{'F1': vf1},epoch_number+1)
        writer.add_scalars('Error Rate',{'error_rate': ver},epoch_number+1)
        writer.flush()

        # Track best performance, and save the model's state
        if F1:
            if vf1 > best_f1:
                best_f1 = vf1
                model_path = 'model_{}_{}'.format(timestamp, epoch_number)
                if device == 'cuda':
                    model.cpu()
                torch.save(model.state_dict(), model_path)
                cnt=0
                model.to(device)
        if ER:
            if ver < best_er:
                best_er = ver
                model_path = 'model_{}_{}'.format(timestamp, epoch_number)
                if device == 'cuda':
                    model.cpu()
                torch.save(model.state_dict(), model_path)
                cnt=0
                model.to(device) 
            
        else:
            cnt+=1
        if cnt == patience:
            print('Early Stopping...')
            break

        epoch_number += 1
    

FOLD 1
--------------------------------


135

23

/home/vpoblete/.conda/envs/sedtorch/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


EPOCH 1:


...

23

EPOCH 1:
EPOCH 2:in 0.5514633267014115 valid 0.4994261860847473
EPOCH 3:in 0.49176612209390713 valid 0.45006170868873596
EPOCH 4:in 0.46413622079072175 valid 0.4490959048271179
EPOCH 5:in 0.4491853396097819 valid 0.43374788761138916
EPOCH 6:in 0.43538374967045257 valid 0.3958142399787903
EPOCH 7:in 0.42475723624229433 valid 0.4037809371948242
EPOCH 8:in 0.41590543036107663 valid 0.39037424325942993
EPOCH 9:in 0.40987669030825297 valid 0.39667242765426636
EPOCH 10:n 0.4013515728491324 valid 0.3766448497772217
EPOCH 11:n 0.3950276716991707 valid 0.3770272135734558
EPOCH 12:n 0.39186642523165105 valid 0.3983910381793976
EPOCH 13:n 0.3840641500773253 valid 0.3876056671142578
EPOCH 14:n 0.38004583848847284 valid 0.3704462945461273
EPOCH 15:n 0.37378819253709583 valid 0.36815696954727173
EPOCH 16:n 0.37191933371402597 valid 0.3576933741569519
EPOCH 17:n 0.3669752776622772 valid 0.3701481819152832
EPOCH 18:n 0.36507018804550173 valid 0.365184485912323
EPOCH 19:n 0.36183350726410196 valid 0.36

EPOCH 149: 0.20877770086129507 valid 0.43349918723106384
EPOCH 150: 0.2079781703375004 valid 0.4212862551212311
EPOCH 151: 0.20939400030506983 valid 0.4343239963054657
EPOCH 152: 0.20955651512852422 valid 0.4377239942550659
EPOCH 153: 0.21119472362377026 valid 0.4579254686832428
EPOCH 154: 0.20698347036485318 valid 0.44151344895362854
EPOCH 155: 0.20604097810056474 valid 0.4359484314918518
EPOCH 156: 0.20573816718878568 valid 0.45590829849243164
EPOCH 157: 0.20578210872632485 valid 0.43772321939468384
EPOCH 158: 0.2048959255218506 valid 0.4499173164367676
EPOCH 159: 0.2082437175291556 valid 0.4511314928531647
EPOCH 160: 0.20414037296065576 valid 0.4466882050037384
EPOCH 161: 0.2061007763500567 valid 0.45030876994132996
EPOCH 162: 0.2060331107289703 valid 0.43011611700057983
EPOCH 163: 0.2076988266574012 valid 0.4292029142379761
EPOCH 164: 0.20362658346140827 valid 0.4309607148170471
EPOCH 165: 0.20870471784362085 valid 0.4434800446033478
EPOCH 166: 0.20374365526217 valid 0.434490710496

EPOCH 296: 0.1671528379674311 valid 0.5158094167709351
EPOCH 297: 0.169395009290289 valid 0.4940464198589325
EPOCH 298: 0.16854045727738629 valid 0.5277431607246399
EPOCH 299: 0.16816026347654836 valid 0.5288378000259399
EPOCH 300: 0.16704504511974477 valid 0.5155346393585205
EPOCH 301: 0.16718578851885266 valid 0.5363548994064331
EPOCH 302: 0.1677834673060311 valid 0.5107415914535522
EPOCH 303: 0.16857015987237295 valid 0.5196207165718079
EPOCH 304: 0.16485381634147078 valid 0.5291259288787842
EPOCH 305: 0.16487265080213548 valid 0.5338963866233826
EPOCH 306: 0.16355669338394094 valid 0.5291301012039185
EPOCH 307: 0.1760737669136789 valid 0.5174755454063416
EPOCH 308: 0.1673476924498876 valid 0.532954752445221
EPOCH 309: 0.1646549235339518 valid 0.5166043639183044
EPOCH 310: 0.1660022449714166 valid 0.5209571123123169
EPOCH 311: 0.16248938259151247 valid 0.5444099307060242
EPOCH 312: 0.16693542257503227 valid 0.5342960953712463
EPOCH 313: 0.1667790306387124 valid 0.5117307305335999
EP

EPOCH 444: 0.15066778185190977 valid 0.5860040187835693
EPOCH 445: 0.14614375079119646 valid 0.6107913255691528
EPOCH 446: 0.14469625138574177 valid 0.6092546582221985
EPOCH 447: 0.1498945713595108 valid 0.5656097531318665
EPOCH 448: 0.14914655243908917 valid 0.5760142803192139
EPOCH 449: 0.15627057944182995 valid 0.58207768201828
EPOCH 450: 0.1453674073020617 valid 0.6106696724891663
EPOCH 451: 0.14578262986960233 valid 0.5981045365333557
EPOCH 452: 0.14742895006029694 valid 0.5893856883049011
EPOCH 453: 0.14743920895788404 valid 0.5781267285346985
EPOCH 454: 0.1454064726277634 valid 0.6305049657821655
EPOCH 455: 0.14433832201692792 valid 0.6080261468887329
EPOCH 456: 0.1462574757359646 valid 0.6008860468864441
EPOCH 457: 0.1434755390992871 valid 0.5937836766242981
EPOCH 458: 0.14499478588501613 valid 0.5809162259101868
EPOCH 459: 0.1469653245475557 valid 0.6020320057868958
EPOCH 460: 0.14854415737920337 valid 0.582425057888031
EPOCH 461: 0.1453389098798787 valid 0.5633142590522766
EP